In [7]:
import requests
import pandas as pd

# Replace 'YOUR_API_KEY' with your actual Census API key
census_api = 'a5fa19f53f218542d9d280b933025397c3699424'
api_key = census_api

# Specify the dataset and variables for income and population
income_dataset = 'acs/acs5/profile'  # American Community Survey 5-Year Profiles
population_dataset = 'acs/acs5'  # American Community Survey 5-Year Estimates

income_vars = 'NAME,DP03_0051E,DP03_0052E,DP03_0053E,DP03_0054E,DP03_0055E,DP03_0056E,DP03_0057E,DP03_0058E,DP03_0059E,DP03_0060E,DP03_0061E'  # Variables for income brackets
population_var = 'B01001_001E'  # Variable for total population

# California's FIPS code
for_state = '06'

# Years range from 2015 to 2022
years = range(2015, 2023)

# Create an empty list to store the data
data_list = []

for year in years:
    # Make the API request for income data
    income_base_url = f'https://api.census.gov/data/{year}/{income_dataset}?get={income_vars}&for=place:*&in=state:{for_state}&key={api_key}'
    income_response = requests.get(income_base_url)

    # Make the API request for population data
    population_base_url = f'https://api.census.gov/data/{year}/{population_dataset}?get={population_var}&for=place:*&in=state:{for_state}&key={api_key}'
    population_response = requests.get(population_base_url)

    if income_response.status_code == 200 and population_response.status_code == 200:
        income_data = income_response.json()
        population_data = population_response.json()

        # Add retrieved data to the list
        for entry_income, entry_population in zip(income_data[1:], population_data[1:]):  # Skip the header row
            city_name = entry_income[0]
            income_brackets = entry_income[1:-2]  # Adjust based on the number of income brackets
            population = entry_population[1]

            # Create a dictionary with income and population data
            data_dict = {'City': city_name, 'Population': population, 'Year': year}
            for i in range(len(income_brackets)):
                data_dict[f'Income_{i+1}'] = income_brackets[i]

            data_list.append(data_dict)
    else:
        print(f"Failed to retrieve data for year {year}.")

# Create a DataFrame from the collected data
census_data = pd.DataFrame(data_list)

Failed to retrieve data for year 2022.


In [8]:
census_data.head()

,City,Population,Year,Income_1,Income_2,Income_3,Income_4,Income_5,Income_6,Income_7,Income_8,Income_9,Income_10,Income_11
0,"Camp Nelson CDP, California",06,2015,51,0,0,0,16,0,0,15,0,20,0
1,"Campo CDP, California",06,2015,874,25,79,115,50,127,190,79,147,62,0
2,"Camptonville CDP, California",06,2015,93,10,17,11,10,14,15,0,13,0,3
3,"Canby CDP, California",06,2015,68,0,0,0,0,56,0,0,12,0,0
4,"Carmet CDP, California",06,2015,7,0,0,0,7,0,0,0,0,0,0


In [15]:
column_mapping = {
    'Population': 'FIPS code',
    'Income_1': 'Total Households',
    'Income_2': 'Less than 10,000',
    'Income_3': '10,000 to 14,999',
    'Income_4': '15,000 to 24,999',
    'Income_5': '25,000 to 34,999',
    'Income_6': '35,000 to 49,999',
    'Income_7': '50,000 to 74,999',
    'Income_8': '75,000 to 99,999',
    'Income_9': '100,000 to 149,999',
    'Income_10': '150,000 to 199,999',
    'Income_11': '200,000 or more'
}

In [17]:
census_data.rename(columns=column_mapping, inplace=True)

In [18]:
census_data

,City,FIPS code,Year,Total Households,"Less than 10,000","10,000 to 14,999","15,000 to 24,999","25,000 to 34,999","35,000 to 49,999","50,000 to 74,999","75,000 to 99,999","100,000 to 149,999","150,000 to 199,999","200,000 or more"
0,"Camp Nelson CDP, California",06,2015,51,0,0,0,16,0,0,15,0,20,0
1,"Campo CDP, California",06,2015,874,25,79,115,50,127,190,79,147,62,0
2,"Camptonville CDP, California",06,2015,93,10,17,11,10,14,15,0,13,0,3
3,"Canby CDP, California",06,2015,68,0,0,0,0,56,0,0,12,0,0
4,"Carmet CDP, California",06,2015,7,0,0,0,7,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10825,"Yreka city, California",06,2021,3210,201,276,480,256,544,483,294,381,221,74
10826,"Yuba City city, California",06,2021,23398,987,987,1785,1776,3461,4414,3139,3596,1439,1814
10827,"Yucaipa city, California",06,2021,18197,1139,543,1259,1506,1441,3166,2061,3578,2122,1382
10828,"Yucca Valley town, California",06,2021,8567,385,509,916,1005,1281,1148,906,1433,686,298


In [20]:
census_data.to_csv('california_income_by_city_2015_2021.csv', encoding='utf-8', index=False)